In [ ]:
from map_util import *
from play import GamePlayer
from time import sleep

In [ ]:
def fixed():
    with GamePlayer() as player:
        player.start(6)
        #sequence = 'WNWNWNWNNEEESSSSSSSSSSSSSSSSSSSSSSSSEEEEEEEEEEEEEEEEEENNEEEEEeeeeESSSSSSeEE'
        sequence = 'WNWNWNWNNEEEEEEESWSWSWSSSSSSSSSSSSSSSSSSSSSEEEEEEEEEEEEEEEEENENEEeeeeeeESSSSSSEEeEE'
        #sequence = 'WNWNWNWNNEEESSSSSSSSSSSSSSSSSSSSSSSSEEEEEEEEEEEEEEEEEENNEEESNSNSNS'

        for c in sequence:
            if c == 'N': player.move('N')
            if c == 'E': player.move('E')
            if c == 'S': player.move('S')
            if c == 'W': player.move('W')
            if c == 'n': player.use('N')
            if c == 'e': player.use('E')
            if c == 's': player.use('S')
            if c == 'w': player.use('W')
            
            if player.finished:
                print('FINISHED')
                break
            #sleep(0.1)

#fixed()

In [ ]:
def get_direction(from_pos, to_pos, distances, paths):
    if to_pos in distances:
        return get_direction_towards(paths, from_pos, to_pos)
    
    north = (to_pos[0]-1, to_pos[1])
    south = (to_pos[0]+1, to_pos[1])
    west = (to_pos[0], to_pos[1]-1)
    east = (to_pos[0], to_pos[1]+1)
    
    if from_pos == north: return 'S'
    if from_pos == south: return 'N'
    if from_pos == west: return 'E'
    if from_pos == east: return 'W'

    min_dist = np.inf
    adjacent_pos = None
    if north in distances and distances[north] < min_dist:
        min_dist = distances[north]
        adjacent_pos = north
    if south in distances and distances[south] < min_dist:
        min_dist = distances[south]
        adjacent_pos = south
    if east in distances and distances[east] < min_dist:
        min_dist = distances[east]
        adjacent_pos = east
    if west in distances and distances[west] < min_dist:
        min_dist = distances[west]
        adjacent_pos = west

    return get_direction_towards(paths, from_pos, adjacent_pos)

In [ ]:
def valid_pos(pos):
    return pos is not None and pos[0] >= 0 and pos[1] >= 0

def can_act1():
    global player
    global remain_on_plate_counter
    return valid_pos(player.player1_pos) and player.action1 is None and remain_on_plate_counter <= 0

def can_act2():
    global player
    return valid_pos(player.player2_pos) and player.action2 is None    

def move_to_1(pos):
    global player
    dir = get_direction(player.player1_pos, pos, player.player1_distances, player.player1_paths)
    player.queue_move1(dir)

def move_to_2(pos):
    global player
    dir = get_direction(player.player2_pos, pos, player.player2_distances, player.player2_paths)
    player.queue_move2(dir)

def use_1(pos):
    global player
    if player.player1_pos[0] < pos[0]: player.queue_use1('S')
    if player.player1_pos[0] > pos[0]: player.queue_use1('N')
    if player.player1_pos[1] < pos[1]: player.queue_use1('E')
    if player.player1_pos[1] > pos[1]: player.queue_use1('W')

def use_2(pos):
    global player
    if player.player2_pos[0] < pos[0]: player.queue_use2('S')
    if player.player2_pos[0] > pos[0]: player.queue_use2('N')
    if player.player2_pos[1] < pos[1]: player.queue_use2('E')
    if player.player2_pos[1] > pos[1]: player.queue_use2('W')


In [ ]:
def get_adjacent_pos(pos, distances):
    top = (pos[0]-1, pos[1])
    bottom = (pos[0]+1, pos[1])
    left = (pos[0], pos[1]-1)
    right = (pos[0], pos[1]+1)
    min_dist = np.inf
    min_pos = None
    if top in distances and distances[top] < min_dist:
        min_dist = distances[top]
        min_pos = top
    if bottom in distances:
        min_dist = distances[bottom]
        min_pos = bottom
    if left in distances:
        min_dist = distances[left]
        min_pos = left
    if right in distances:
        min_dist = distances[right]
        min_pos = right
    return min_pos

def are_adjacent(pos1, pos2):
    dy = pos2[0] - pos1[0]
    dx = pos2[1] - pos1[1]
    return (abs(dy) == 1 and dx == 0) or (dy == 0 and abs(dx) == 1)
    
def pickup_key_or_open_door(key, door, item):
    global player
    doors = np.argwhere(player.map == door)
    if np.any(doors):
        door_pos = tuple(doors[0])
        if can_act1() and player.player1_inventory == item:
            if are_adjacent(player.player1_pos, door_pos):
                print('use_door1')
                use_1(door_pos)
            else:
                print('move_door1')
                move_to_1(door_pos)
        elif can_act2() and player.player2_inventory == item:
            if are_adjacent(player.player2_pos, door_pos):
                print('use_door2')
                use_2(door_pos)
            else:
                print('move_door2')
                move_to_2(door_pos)
        else:
            keys = np.argwhere(player.map == key)
            if np.any(keys):
                
                if can_act1() and player.player1_inventory == 0:
                    for key in keys:
                        key_pos = tuple(key)
                        if get_direction(player.player1_pos, key_pos, player.player1_distances, player.player1_paths) is not None:
                            print('move_key1')
                            move_to_1(key_pos)
                            break

                if can_act2() and player.player2_inventory == 0:
                    for key in keys:
                        key_pos = tuple(key)
                        if get_direction(player.player2_pos, key_pos, player.player2_distances, player.player2_paths) is not None:
                            print('move_key2')
                            move_to_2(key_pos)
                            break


In [ ]:
def get_dist_to_other_player(player1_pos, player2_pos, player1_distances):
    if not valid_pos(player1_pos): return None
    if not valid_pos(player2_pos): return None
    
    if are_adjacent(player1_pos, player2_pos): return 1
    
    adj_pos = get_adjacent_pos(player2_pos, player1_distances)
    if adj_pos is None: return None

    dist = player1_distances[adj_pos] + 1
    return dist

In [ ]:
remain_on_plate_counter = 0

with GamePlayer() as player:
    player.start(9)

    while not player.finished:
        
        # Move to exit if possible
        exits = np.argwhere(player.map == EXIT)
        if np.any(exits):
            exit_pos = tuple(exits[0])
            if can_act1():
                print('exit1')
                move_to_1(exit_pos)
            if can_act2():
                print('exit2')
                move_to_2(exit_pos)

        # Attack
        enemies = np.argwhere(player.map == ENEMY)
        if np.any(enemies):
            enemy_pos = tuple(enemies[0])
            
            # make sure players are close to each other so they can both attack
            dist_1_to_2 = get_dist_to_other_player(player.player1_pos, player.player2_pos, player.player1_distances)
            dist_2_to_1 = get_dist_to_other_player(player.player2_pos, player.player1_pos, player.player2_distances)
            if dist_1_to_2 is None and dist_2_to_1 is None:
                dist_players = None
            elif dist_1_to_2 is None:
                dist_players = dist_2_to_1
            elif dist_2_to_1 is None:
                dist_players = dist_1_to_2
            else:
                dist_players = min(dist_1_to_2, dist_2_to_1)
            print(f'dist: 1>2: {dist_1_to_2}, 2>1: {dist_2_to_1}')
            
            if can_act1() and player.player1_has_sword:
                can_attack = True
                if dist_players is not None and dist_players > 5:
                    print('move_closer1')
                    move_to_1(player.player2_pos)
                    can_attack = False
                
                if can_attack:
                    if are_adjacent(player.player1_pos, enemy_pos):
                        print('attack_use1')
                        use_1(enemy_pos)
                    else:
                        print('attack_move1')
                        move_to_1(enemy_pos)

            if can_act2() and player.player2_has_sword:
                can_attack = True
                if dist_players is not None and dist_players > 4:
                    print('move_closer2')
                    move_to_2(player.player1_pos)
                    can_attack = False

                if can_attack:
                    if are_adjacent(player.player2_pos, enemy_pos):
                        print('attack_use2')
                        use_2(enemy_pos)
                    else:
                        print('attack_move2')
                        move_to_2(enemy_pos)

        # Pickup health
        healths = np.argwhere(player.map == HEALTH)
        if np.any(healths):
            health_pos = tuple(healths[0])
            if can_act1() and player.player1_health < 5:
                print('health1')
                move_to_1(health_pos)
            elif can_act2() and player.player2_health < 5:
                print('health2')
                move_to_2(health_pos)

        # Pickup sword
        swords = np.argwhere(player.map == SWORD)
        if np.any(swords):
            sword_pos = tuple(swords[0])
            if can_act1() and not player.player1_has_sword:
                print('sword1')
                move_to_1(sword_pos)
            elif can_act2() and not player.player2_has_sword:
                print('sword2')
                move_to_2(sword_pos)
                
        # Pickup keys
        pickup_key_or_open_door(KEY_RED, DOOR_RED, INVENTORY_KEY_RED)
        pickup_key_or_open_door(KEY_GREEN, DOOR_GREEN, INVENTORY_KEY_GREEN)
        pickup_key_or_open_door(KEY_BLUE, DOOR_BLUE, INVENTORY_KEY_BLUE)

        # Explore
        if can_act1():
            dir = player.get_direction_towards_closest_unknown(player.player1_pos, player.player1_distances, player.player1_paths)
            if dir is not None:
                print('explore1')
                player.queue_move1(dir)
        if can_act2():
            dir = player.get_direction_towards_closest_unknown(player.player2_pos, player.player2_distances, player.player2_paths)
            if dir is not None:
                print('explore2')
                player.queue_move2(dir)
        
        plates = np.argwhere(player.map == PRESSURE_PLATE)
        plate_pos = None
        if np.any(plates):
            plate_pos = tuple(plates[0])
            if can_act1():
                print('plate1')
                move_to_1(plate_pos)

        black_doors = np.argwhere(player.map == DOOR_BLACK)
        if np.any(black_doors):
            black_door_pos = tuple(black_doors[0])
            if can_act2():
                print('move_black2')
                move_to_2(black_door_pos)            
        
        player.act()
        
        if plate_pos is not None and player.player1_pos[0] == plate_pos[0] and player.player1_pos[1] == plate_pos[1]:
            print(f'Reset {remain_on_plate_counter=}')
            remain_on_plate_counter = 100
        remain_on_plate_counter -= 1
        
        #player.step()
        #sleep(0.2)